# ActivateBaby - buliding tokenizer
based on [How to train a new language model from scratch using Transformers and Tokenizers](https://colab.research.google.com/github/huggingface/blog/blob/main/notebooks/01_how_to_train.ipynb#scrollTo=M1oqh0F6W3ad)


# 0. init

In [2]:
conda env list

# conda environments:
#
base                     /root/miniconda3
CreativeSumm             /root/miniconda3/envs/CreativeSumm
allennlp_env             /root/miniconda3/envs/allennlp_env
codenames                /root/miniconda3/envs/codenames
rsa_tapm                 /root/miniconda3/envs/rsa_tapm
sum                      /root/miniconda3/envs/sum
tapm                  *  /root/miniconda3/envs/tapm
transformers             /root/miniconda3/envs/transformers


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from os.path import join as osj
from pathlib import Path

import pandas as pd
from tqdm import tqdm

## 1. Dataset

In [5]:
DATA_PATH = '/root/xhong/babylm/babylm_data'

In [6]:
data_version = 'babylm_10M'
data_path = osj(DATA_PATH, data_version)
data_dict = {}

for f in os.listdir(data_path):
    if f[0] == '.':
        continue
    corpus_name = f.split('.')[0]
    print(f)

open_subtitles.train
simple_wikipedia.train
wikipedia.train
switchboard.train
cbt.train
aochildes.train
bnc_spoken.train
qed.train
gutenberg.train
children_stories.train


In [39]:
paths = [str(x) for x in Path(data_path).glob("**/*.train")]

with open('babylm_10M.txt', 'w') as outfile:
    for fname in paths:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

## 2. Train a ByteLevelBPETokenizer tokenizer

We choose to train a byte-level Byte-pair encoding tokenizer (the same as GPT-2), with the same special tokens as RoBERTa. Let’s arbitrarily pick its size to be 52,000.

We recommend training a byte-level BPE (rather than let’s say, a WordPiece tokenizer like BERT) because it will start building its vocabulary from an alphabet of single bytes, so all words will be decomposable into tokens (no more `<unk>` tokens!).


In [7]:
%%time 

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(data_path).glob("**/*.train")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])




CPU times: user 1min 56s, sys: 19.2 s, total: 2min 15s
Wall time: 10.6 s


Now let's save files to disk

In [8]:
!mkdir ABByteLevelBPE
tokenizer.save_model("ABByteLevelBPE")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['ABByteLevelBPE/vocab.json', 'ABByteLevelBPE/merges.txt']

🔥🔥 Wow, that was fast! ⚡️🔥

We now have both a `vocab.json`, which is a list of the most frequent tokens ranked by frequency, and a `merges.txt` list of merges.

```json
{
	"<s>": 0,
	"<pad>": 1,
	"</s>": 2,
	"<unk>": 3,
	"<mask>": 4,
	"!": 5,
	"\"": 6,
	"#": 7,
	"$": 8,
	"%": 9,
	"&": 10,
	"'": 11,
	"(": 12,
	")": 13,
	# ...
}

# merges.txt
l a
Ġ k
o n
Ġ la
t a
Ġ e
Ġ d
Ġ p
# ...
```

What is great is that our tokenizer is optimized for Esperanto. Compared to a generic tokenizer trained for English, more native words are represented by a single, unsplit token. Diacritics, i.e. accented characters used in Esperanto – `ĉ`, `ĝ`, `ĥ`, `ĵ`, `ŝ`, and `ŭ` – are encoded natively. We also represent sequences in a more efficient manner. Here on this corpus, the average length of encoded sequences is ~30% smaller as when using the pretrained GPT-2 tokenizer.

Here’s  how you can use it in `tokenizers`, including handling the RoBERTa special tokens – of course, you’ll also be able to use it directly from `transformers`.


In [9]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./ABByteLevelBPE/vocab.json",
    "./ABByteLevelBPE/merges.txt",
)

In [10]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [21]:
tokenizer.encode("Hello world").ids

[0, 2230, 1019, 2]

In [23]:
tokenizer.encode(" Hello world").ids

[0, 6168, 1019, 2]

In [22]:
tokenizer.encode("Hello world").tokens

['<s>', 'Hello', 'Ġworld', '</s>']

In [24]:
tokenizer.encode(" Hello world").tokens

['<s>', 'ĠHello', 'Ġworld', '</s>']

## 2b. Train a LongformerTokenizer

In [49]:
%%time 

from tokenizers import LongformerTokenizer

paths = [str(x) for x in Path(data_path).glob("**/*.train")]

# Initialize a tokenizer
tokenizer = LongformerTokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

ImportError: cannot import name 'LongformerTokenizer' from 'tokenizers' (/root/miniconda3/envs/tapm/lib/python3.7/site-packages/tokenizers/__init__.py)

In [8]:
!mkdir ABByteLevelBPE
tokenizer.save_model("ABByteLevelBPE")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['ABByteLevelBPE/vocab.json', 'ABByteLevelBPE/merges.txt']

In [9]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./ABByteLevelBPE/vocab.json",
    "./ABByteLevelBPE/merges.txt",
)

In [10]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [21]:
tokenizer.encode("Hello world").ids

[0, 2230, 1019, 2]

In [23]:
tokenizer.encode(" Hello world").ids

[0, 6168, 1019, 2]

In [22]:
tokenizer.encode("Hello world").tokens

['<s>', 'Hello', 'Ġworld', '</s>']

In [24]:
tokenizer.encode(" Hello world").tokens

['<s>', 'ĠHello', 'Ġworld', '</s>']

## 3. Train a language model from scratch


### Now let's build our training Dataset

We'll build our dataset by applying our tokenizer to our text file.

Here, as we only have one text file, we don't even need to customize our `Dataset`. We'll just use the `LineByLineDataset` out-of-the-box.

In [40]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./babylm_10M.txt",
    block_size=128,
)

2023-05-17 03:57:05.143979: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-17 03:57:05.144012: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/root/miniconda3/envs/tapm/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 2min 10s, sys: 8.39 s, total: 2min 18s
Wall time: 47.8 s


Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [42]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)